# **Step 1.** Install Python Packages

In [1]:
!pip install -U xformers --index-url https://download.pytorch.org/whl/cu121
!pip install --no-deps packaging ninja einops flash-attn trl peft accelerate bitsandbytes
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

!pip install torchvision==0.21.0

!pip install --upgrade transformers bitsandbytes unsloth

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 99.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 124.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/

Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/torchvision-0.21.0+cu124.dist-info/*
    /usr/local/lib/python3.11/dist-packages/torchvision.libs/libcudart.41118559.so.12
    /usr/local/lib/python3.11/dist-packages/torchvision.libs/libjpeg.1c1c4b09.so.8
    /usr/local/lib/python3.11/dist-packages/torchvision.libs/libnvjpeg.02b6d700.so.12
    /usr/local/lib/python3.11/dist-packages/torchvision.libs/libpng16.0364a1db.so.16
    /usr/local/lib/python3.11/dist-packages/torchvision.libs/libsharpyuv.5c41a003.so.0
    /usr/local/lib/python3.11/dist-packages/torchvision.libs/libwebp.54a0d02a.so.7
    /usr/local/lib/python3.11/dist-packages/torchvision.libs/libz.d13a2644.so.1
    /usr/local/lib/python3.11/dist-packages/torchvision/*
Proceed (Y/n)? n
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB

# **Step 2.** Import Python Packages

In [2]:
import unsloth
import torch
import os
import json
import pandas as pd
from datasets import Dataset, DatasetDict
from datasets import load_dataset
from huggingface_hub import notebook_login
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.


    PyTorch 2.5.1+cu121 with CUDA 1201 (you have 2.6.0+cu124)
    Python  3.11.11 (you have 3.11.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


# **Step 3.** Login to Your Hugging Face with hf_token. (write access token)

In [3]:
notebook_login()

# **Step 4.** Convert your JSON dataset to Llama3 finetuning format


In [ ]:
huggingface_user = "kriteekon"
dataset_name = "aave_matched_indirect"

class Llama3InstructDataset:
    def __init__(self, data):
        self.data = data
        self.prompts = []
        self.create_prompts()

    def create_prompt(self, row):
        prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>{row['instruction']}<|eot_id|><|start_header_id|>user<|end_header_id|>{row['input']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>{row['output']}<|eot_id|>"""
        return prompt

    def create_prompts(self):
        for row in self.data:
            prompt = self.create_prompt(row)
            self.prompts.append(prompt)

    def get_dataset(self):
        df = pd.DataFrame({'prompt': self.prompts})
        return df

def create_dataset_hf(dataset):
    dataset.reset_index(drop=True, inplace=True)
    return DatasetDict({"train": Dataset.from_pandas(dataset)})

if __name__ == "__main__":
    # 1) Load both JSON files
    with open('/content/training_files_finetune.json','r') as f:
        train_data = json.load(f)
    with open('/content/validation_files_finetune.json','r') as f:
        val_data   = json.load(f)

    # 2) Build prompts for each split
    train_ds = Llama3InstructDataset(train_data).get_dataset()
    val_ds   = Llama3InstructDataset(val_data).get_dataset()

    # 3) Wrap into a DatasetDict with train + validation
    from datasets import Dataset, DatasetDict
    hf_datasets = DatasetDict({
        "train":      Dataset.from_pandas(train_ds),
        "validation": Dataset.from_pandas(val_ds),
    })

    # 4) (Optional) save locally
    processed_data_path = 'processed_data/llama3_dataset'
    hf_datasets.save_to_disk(processed_data_path)

    # 5) Push *both* splits to the Hub in one repo
    hf_datasets.push_to_hub(f"{huggingface_user}/{dataset_name}")

Saving the dataset (0/1 shards):   0%|          | 0/1376 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/172 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/390 [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


# **Step 5.** LoRa Finetuning Configurations
- "finetuned_model" sets your models name on HF
- "num_train_epochs" sets the number of epochs for training

    (epoch = 1 pass through your entire dataset)

In [ ]:
# Defining the configuration for the base model, LoRA and training
# Defining the configuration for the base model, LoRA and training
config = {
    "hugging_face_username": huggingface_user,
    "model_config": {
        "base_model":      "meta-llama/Llama-3.1-8B-Instruct",
        "finetuned_model": f"{huggingface_user}/llama-3.1-8b-instruct-mynewmodel",
        "max_seq_length":  2048,
        "dtype":           torch.float16,
        "load_in_4bit":    True,
    },
        "lora_config": {
        "r": 8,  # Use 8 based on best hyperparameters
        "target_modules": [
            "q_proj",
            "k_proj",
            "v_proj",
        ],  # Best modules you found
        "lora_alpha": 8,  # Match alpha to r (usually good practice)
        "lora_dropout": 0.2,  # Best dropout you found
        "bias": "none",  # Keep same
        "use_gradient_checkpointing": True,  # Good to keep for memory savings
        "use_rslora": False,  # Keep off
        "use_dora": False,    # Keep off
        "loftq_config": None, # Keep None
    },

    "training_dataset": {
        "name": f"{huggingface_user}/{dataset_name}",  # The dataset name(huggingface/datasets)
        "split": "train",  # The dataset split
        "input_field": "prompt",  # The input field
    },
    "eval_dataset": {
        "name": f"{huggingface_user}/{dataset_name}",  # same dataset repo
        "split": "validation",                         # use the validation split
        "input_field": "prompt",                       # field with prompt text
    },
    "training_config": {
        "per_device_train_batch_size": 2,
        "gradient_accumulation_steps": 1,    # update: log & step every micro‑batch
        "warmup_steps": 50,                  # a bit more warmup
        "max_steps": -1,                     # disable max_steps, use epochs
        "num_train_epochs": 3,               # three full passes over your data
        "learning_rate": 2e-5,               # lower LR for stable tuning
        "fp16": True,
        "bf16": False,
        "logging_strategy": "steps",         # log by step count
        "logging_steps": 10,                 # log every 10 steps
        "logging_first_step": True,          # also log step 1
        "save_strategy": "epoch",            # save checkpoint each epoch
        "evaluation_strategy": "epoch",      # eval each epoch (if you have an eval set)
        "optim": "adamw_torch",              # switch off 8‑bit AdamW if unstable
        "weight_decay": 0.01,
        "lr_scheduler_type": "linear",
        "seed": 42,
        "output_dir": "outputs",
    },
}

# **Step 6.** Load Llama3-8B, QLoRA & Trainer Model

In [1]:
#DONT NEED THIS
# from transformers import TrainingArguments

# # Loading the model and the tokinizer for the model
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = config.get("model_config").get("base_model"),
#     max_seq_length = config.get("model_config").get("max_seq_length"),
#     dtype = config.get("model_config").get("dtype"),
#     load_in_4bit = config.get("model_config").get("load_in_4bit"),
# )

# # Setup for QLoRA/LoRA peft of the base model
# model = FastLanguageModel.get_peft_model(
#     model,
#     r = config.get("lora_config").get("r"),
#     target_modules = config.get("lora_config").get("target_modules"),
#     lora_alpha = config.get("lora_config").get("lora_alpha"),
#     lora_dropout = config.get("lora_config").get("lora_dropout"),
#     bias = config.get("lora_config").get("bias"),
#     use_gradient_checkpointing = config.get("lora_config").get("use_gradient_checkpointing"),
#     random_state = 42,
#     use_rslora = config.get("lora_config").get("use_rslora"),
#     use_dora = config.get("lora_config").get("use_dora"),
#     loftq_config = config.get("lora_config").get("loftq_config"),
# )

# # Loading the training dataset
# # dataset_train = load_dataset(config.get("training_dataset").get("name"), split = config.get("training_dataset").get("split"))
# # print(len(dataset_train))

# dataset_train = load_dataset(
#     config["training_dataset"]["name"],
#     split=config["training_dataset"]["split"],      # "train"
# )
# print(f"Train size: {len(dataset_train)}")

# # —————————————————————————————————————————
# # Here’s where you add the validation split:
# dataset_val = load_dataset(
#     config["training_dataset"]["name"],
#     split="validation",                             # assumes you pushed a "validation" split
# )
# print(f"Validation size: {len(dataset_val)}")
# # —————————————————————————————————————————

# training_args = TrainingArguments(
#     output_dir                    = config["training_config"]["output_dir"],
#     per_device_train_batch_size  = config["training_config"]["per_device_train_batch_size"],
#     gradient_accumulation_steps  = config["training_config"]["gradient_accumulation_steps"],
#     warmup_steps                 = config["training_config"]["warmup_steps"],
#     max_steps                    = config["training_config"]["max_steps"],
#     num_train_epochs             = config["training_config"]["num_train_epochs"],
#     learning_rate                = config["training_config"]["learning_rate"],
#     fp16                         = config["training_config"]["fp16"],
#     bf16                         = config["training_config"]["bf16"],
#     logging_steps                = config["training_config"]["logging_steps"],
#     logging_first_step           = config["training_config"].get("logging_first_step", True),
#     optim                        = config["training_config"]["optim"],
#     weight_decay                 = config["training_config"]["weight_decay"],
#     lr_scheduler_type            = config["training_config"]["lr_scheduler_type"],
#     seed                         = config["training_config"]["seed"],
#     save_strategy                = "epoch",
#     save_total_limit             = 2,
# )

# # Tokenize the dataset (minimal example)
# def tokenize_function(example):
#     tokens = tokenizer(
#         example[config["training_dataset"]["input_field"]],
#         truncation=True,
#         padding="max_length",
#         max_length=config["model_config"]["max_seq_length"],
#     )
#     tokens["labels"] = tokens["input_ids"].copy()  # <-- this is the key fix!
#     return tokens

# tokenized_train = dataset_train.map(tokenize_function, batched=True)
# tokenized_val   = dataset_val.map(tokenize_function, batched=True)




# **This is for grid search**

In [ ]:
# ✅ Force use of Hugging Face's real Trainer and TrainingArguments
from transformers import TrainingArguments as HFTrainingArguments
from transformers import Trainer as HFTrainer
from transformers import EarlyStoppingCallback


In [ ]:
def run_training_with_config(r, dropout, target_modules):
    print(f"🔧 Running with r={r}, dropout={dropout}, modules={target_modules}")

    config["lora_config"]["r"] = r
    config["lora_config"]["lora_alpha"] = 2 * r
    config["lora_config"]["lora_dropout"] = dropout
    config["lora_config"]["target_modules"] = target_modules

    # Reload model and tokenizer fresh each time
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=config["model_config"]["base_model"],
        max_seq_length=config["model_config"]["max_seq_length"],
        dtype=config["model_config"]["dtype"],
        load_in_4bit=config["model_config"]["load_in_4bit"],
    )

    model = FastLanguageModel.get_peft_model(
        model,
        r=config["lora_config"]["r"],
        target_modules=config["lora_config"]["target_modules"],
        lora_alpha=config["lora_config"]["lora_alpha"],
        lora_dropout=config["lora_config"]["lora_dropout"],
        bias=config["lora_config"]["bias"],
        use_gradient_checkpointing=config["lora_config"]["use_gradient_checkpointing"],
        random_state=42,
        use_rslora=config["lora_config"]["use_rslora"],
        use_dora=config["lora_config"]["use_dora"],
        loftq_config=config["lora_config"]["loftq_config"],
    )

    # Tokenize datasets
    def tokenize_function(example):
        tokens = tokenizer(
            example[config["training_dataset"]["input_field"]],
            truncation=True,
            padding="max_length",
            max_length=config["model_config"]["max_seq_length"],
        )
        tokens["labels"] = tokens["input_ids"].copy()
        return tokens

    tokenized_train = dataset_train.map(tokenize_function, batched=True)
    tokenized_val = dataset_val.map(tokenize_function, batched=True)

    run_name = f"r{r}_drop{int(dropout*100)}_{'-'.join(target_modules)}"
    output_dir = f"{config['training_config']['output_dir']}/{run_name}"

    training_args = HFTrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=config["training_config"]["per_device_train_batch_size"],
        gradient_accumulation_steps=config["training_config"]["gradient_accumulation_steps"],
        warmup_steps=config["training_config"]["warmup_steps"],
        max_steps=config["training_config"]["max_steps"],
        num_train_epochs=config["training_config"]["num_train_epochs"],
        learning_rate=config["training_config"]["learning_rate"],
        fp16=config["training_config"]["fp16"],
        bf16=config["training_config"]["bf16"],
        logging_steps=config["training_config"]["logging_steps"],
        logging_first_step=config["training_config"].get("logging_first_step", True),
        optim=config["training_config"]["optim"],
        weight_decay=config["training_config"]["weight_decay"],
        lr_scheduler_type=config["training_config"]["lr_scheduler_type"],
        seed=config["training_config"]["seed"],
        save_strategy="epoch",  # Save manually at each epoch
        save_total_limit=2,
    )

    trainer = HFTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        tokenizer=tokenizer,
    )

    best_loss = float("inf")
    patience_counter = 0
    max_patience = 2
    EPOCHS = config["training_config"]["num_train_epochs"]

    trainer_stats = []  # <-- ADD THIS

    for epoch in range(EPOCHS):
        print(f"\n🚀 Starting Epoch {epoch + 1}/{EPOCHS} for run: {run_name}")
        trainer.train(resume_from_checkpoint=True if epoch > 0 else None)
        eval_results = trainer.evaluate()
        val_loss = eval_results["eval_loss"]
        print(f"📉 Validation loss after epoch {epoch + 1}: {val_loss:.4f}")

        # Save stats after each epoch
        trainer_stats.append({
            "epoch": epoch + 1,
            "val_loss": val_loss,
        })

        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            print(f"⚠️ No improvement. Patience: {patience_counter}/{max_patience}")

        if patience_counter >= max_patience:
            print(f"🛑 Early stopping triggered at epoch {epoch + 1}")
            break

    # AFTER ALL EPOCHS - Save the trainer stats
    import json
    with open(f"{output_dir}/trainer_stats.json", "w") as f:
        json.dump(trainer_stats, f, indent=4)

    print(f"✅ Saved training stats to {output_dir}/trainer_stats.json")


# Grid Search Metrics

In [ ]:
# from itertools import product

# r_values = [2, 4, 8]
# dropout_values = [0.05, 0.1, 0.2]
# target_modules_list = [
#     ["q_proj", "v_proj"],
#     ["q_proj", "k_proj", "v_proj"]
# ]

# for r, dropout, modules in product(r_values, dropout_values, target_modules_list):
#     run_training_with_config(r, dropout, modules)
run_training_with_config(
    r=8,
    dropout=0.2,
    target_modules=["q_proj", "k_proj", "v_proj"]
)


🔧 Running with r=8, dropout=0.2, modules=['q_proj', 'k_proj', 'v_proj']
==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Map:   0%|          | 0/1474 [00:00<?, ? examples/s]

Map:   0%|          | 0/185 [00:00<?, ? examples/s]

<ipython-input-10-d4e97aa7bf38>:68: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = HFTrainer(
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,474 | Num Epochs = 3 | Total steps = 2,211
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 4,718,592/8,000,000,000 (0.06% trained)



🚀 Starting Epoch 1/3 for run: r8_drop20_q_proj-k_proj-v_proj


...

Step,Training Loss


📉 Validation loss after epoch 2: 7.9201
⚠️ No improvement. Patience: 1/2

🚀 Starting Epoch 3/3 for run: r8_drop20_q_proj-k_proj-v_proj


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,474 | Num Epochs = 3 | Total steps = 2,211
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 4,718,592/8,000,000,000 (0.06% trained)


Step,Training Loss


📉 Validation loss after epoch 3: 7.9201
⚠️ No improvement. Patience: 2/2
🛑 Early stopping triggered at epoch 3
✅ Saved training stats to outputs/r8_drop20_q_proj-k_proj-v_proj/trainer_stats.json


# If running without Grid Search


# **Step 7.** Train Your Finetuned Model

In [ ]:
r = config["lora_config"]["r"]
dropout = config["lora_config"]["lora_dropout"]
target_modules = config["lora_config"]["target_modules"]

config["lora_config"]["r"] = r
config["lora_config"]["lora_alpha"] = 2 * r
config["lora_config"]["lora_dropout"] = dropout
config["lora_config"]["target_modules"] = target_modules

# 2) Load base model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = config["model_config"]["base_model"],
    max_seq_length = config["model_config"]["max_seq_length"],
    dtype          = config["model_config"]["dtype"],
    load_in_4bit   = config["model_config"]["load_in_4bit"],
)

model = FastLanguageModel.get_peft_model(
    model,
    r                         = config["lora_config"]["r"],
    target_modules            = config["lora_config"]["target_modules"],
    lora_alpha                = config["lora_config"]["lora_alpha"],
    lora_dropout              = config["lora_config"]["lora_dropout"],
    bias                      = config["lora_config"]["bias"],
    use_gradient_checkpointing= config["lora_config"]["use_gradient_checkpointing"],
    random_state              = 42,
    use_rslora                = config["lora_config"]["use_rslora"],
    use_dora                  = config["lora_config"]["use_dora"],
    loftq_config              = config["lora_config"]["loftq_config"],
)

# 3) Load processed dataset
dataset = DatasetDict.load_from_disk('processed_data/llama3_dataset')

# 4) Tokenize datasets
def tokenize_function(example):
    out = tokenizer(
        example["prompt"],
        truncation=True,
        padding="max_length",
        max_length=config["model_config"]["max_seq_length"],
    )
    out["labels"] = out["input_ids"].copy()
    return out

tokenized_train = dataset["train"].map(tokenize_function, batched=True, remove_columns=dataset["train"].column_names)
tokenized_val = dataset["validation"].map(tokenize_function, batched=True, remove_columns=dataset["validation"].column_names)

# 5) Set TrainingArguments
run_name = f"r{r}_drop{int(dropout*100)}_{'-'.join(target_modules)}"
output_dir = f"{config['training_config']['output_dir']}/{run_name}"

training_args = TrainingArguments(
    output_dir                  = output_dir,
    per_device_train_batch_size = config["training_config"]["per_device_train_batch_size"],
    gradient_accumulation_steps = config["training_config"]["gradient_accumulation_steps"],
    warmup_steps                = config["training_config"]["warmup_steps"],
    max_steps                   = config["training_config"]["max_steps"],
    num_train_epochs            = config["training_config"]["num_train_epochs"],
    learning_rate               = config["training_config"]["learning_rate"],
    fp16                        = config["training_config"]["fp16"],
    bf16                        = config["training_config"]["bf16"],
    logging_steps               = config["training_config"]["logging_steps"],
    logging_first_step          = config["training_config"].get("logging_first_step", True),
    optim                       = config["training_config"]["optim"],
    weight_decay                = config["training_config"]["weight_decay"],
    lr_scheduler_type           = config["training_config"]["lr_scheduler_type"],
    seed                        = config["training_config"]["seed"],
    # save_strategy               = "epoch",
    # evaluation_strategy         = "epoch",
    save_total_limit            = 2,
    push_to_hub                 = False,
    report_to                   = "none",
)

# 6) Initialize SFTTrainer
trainer = SFTTrainer(
    model         = model,
    args          = training_args,
    train_dataset = tokenized_train,
    eval_dataset  = tokenized_val,      # ✅ YES pass validation set here
    tokenizer     = tokenizer,
    packing       = False,
    dataset_text_field = "prompt",
)


# 7) Train and Evaluate manually after each epoch

best_val_loss = float('inf')
patience = 0
max_patience = 2
stats = []

for epoch in range(int(config["training_config"]["num_train_epochs"])):
    print(f"🚀 Starting Epoch {epoch+1}")

    trainer.train(resume_from_checkpoint=(epoch > 0))

    print(f"📉 Running evaluation after epoch {epoch+1}")
    results = trainer.evaluate()
    val_loss = results["eval_loss"]
    print(f"✅ Validation Loss: {val_loss:.4f}")

    stats.append({"epoch": epoch+1, "val_loss": val_loss})

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience = 0
    else:
        patience += 1
        print(f"⚠️ No improvement. Patience {patience}/{max_patience}")
        if patience >= max_patience:
            print("🛑 Early stopping triggered")
            break

# 8) Save evaluation results
os.makedirs(output_dir, exist_ok=True)
with open(f"{output_dir}/trainer_stats.json", "w") as f:
    json.dump(stats, f, indent=4)

print(f"✅ Stats saved to {output_dir}/trainer_stats.json")


==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Map:   0%|          | 0/172 [00:00<?, ? examples/s]

🚀 Starting Epoch 1


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,376 | Num Epochs = 3 | Total steps = 2,064
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 4,718,592/8,000,000,000 (0.06% trained)


Step,Training Loss
1,2.348800
10,2.350300
20,2.353600
30,2.331800
40,2.293800
50,2.181300
60,2.031600
70,1.897300
80,1.752600
90,1.597600


...

Step,Training Loss


📉 Running evaluation after epoch 2
✅ Validation Loss: 0.3933
🚀 Starting Epoch 3


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,376 | Num Epochs = 3 | Total steps = 2,064
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 4,718,592/8,000,000,000 (0.06% trained)


Step,Training Loss


📉 Running evaluation after epoch 3
✅ Validation Loss: 0.3933
⚠️ No improvement. Patience 1/2
✅ Stats saved to outputs/r8_drop20_q_proj-k_proj-v_proj/trainer_stats.json


# **Step 8.** Save Finetuned Model & Push to HF Hub

In [ ]:
# trainer.model.save_pretrained("llama-3.1-8b-instruct-4bit-qLoRA-mynewdata")
# tokenizer.save_pretrained("llama-3.1-8b-instruct-4bit-qLoRA-mynewdata")
model.save_pretrained("llama-3.1-8b-instruct-mynewmodel")
tokenizer.save_pretrained("llama-3.1-8b-instruct-mynewmodel")


# 2) Push *that* directory to HF Hub
model.push_to_hub(
    repo_id="kriteekon/llama-3.1-8b-instruct-mynewmodel",
    use_auth_token=True,
)



  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/9.46M [00:00<?, ?B/s]

Saved model to https://huggingface.co/kriteekon/llama-3.1-8b-instruct-mynewmodel


# **Step 10.** Test your pretrained model in Colab

In [6]:
import sys
import json
import torch
import re
from unsloth import FastLanguageModel

# ——————————————————————————————
# Configuration
# ——————————————————————————————

HF_USER   = "kriteekon"
MODEL_NAME = f"{HF_USER}/llama-3.1-8b-instruct-mynewmodel"
TEST_JSON  = "/content/sae_files_finetune_no_outputs.json"
OUTPUT_TXT = "generated_output_indirect_SAE_3.1_updated.txt"

# The 12 characteristics in order
CHARACTERISTICS = [
    "Intelligence", "Determination", "Calmness", "Politeness",
    "Aggression", "Sophistication", "Incoherence", "Rudeness",
    "Stupidity", "Articulation", "Unsophistication", "Laziness"
]

# ——————————————————————————————
# Helpers
# ——————————————————————————————

class Tee(object):
    """Duplicate stdout to multiple file-like objects."""
    def __init__(self, *files):
        self.files = files
    def write(self, obj):
        for f in self.files:
            f.write(obj)
            f.flush()
    def flush(self):
        for f in self.files:
            f.flush()

def extract_scores(response: str) -> tuple[dict, dict]:
    pairs = re.findall(r'([A-Za-z]+):\s*([1-5])', response)
    if len(pairs) >= 24:
        p1 = {CHARACTERISTICS[i]: int(pairs[i][1])     for i in range(12)}
        p2 = {CHARACTERISTICS[i]: int(pairs[i+12][1])  for i in range(12)}
        return p1, p2
    if len(pairs) >= 12:
        p1 = {CHARACTERISTICS[i]: int(pairs[i][1]) for i in range(12)}
        return p1, {}
    return {}, {}

def process_test_json(input_file, output_file, model, tokenizer, max_new_tokens=256):
    try:
        data = json.load(open(input_file, "r", encoding="utf-8"))
    except Exception as e:
        print("❌ Failed to load JSON:", e)
        return

    with open(output_file, "w", encoding="utf-8") as out:
        # redirect all prints to both console and file
        original_stdout = sys.stdout
        sys.stdout = Tee(sys.stdout, out)
        try:
            for entry in data:
                instr = entry.get("instruction", "").strip()
                ui    = entry.get("input", "").strip()
                if not instr or not ui:
                    print("⚠️ Skipping invalid entry.")
                    continue

                bos = tokenizer.bos_token
                eos = tokenizer.eos_token
                prompt = f"{bos} system: {instr} {eos}\nuser: {ui} {eos}"

                print("PROMPT:\n", prompt, "\n---")
                inputs = tokenizer([prompt], return_tensors="pt")
                device = "cuda" if torch.cuda.is_available() else "cpu"
                inputs = {k: v.to(device) for k, v in inputs.items()}

                with torch.no_grad():
                    outputs = model.generate(
                        **inputs,
                        max_new_tokens=max_new_tokens,
                        do_sample=False,
                        temperature=0.0,
                        top_k=1,
                        top_p=1.0,
                        eos_token_id=tokenizer.eos_token_id,
                        pad_token_id=tokenizer.eos_token_id,
                    )

                response = tokenizer.decode(outputs[0], skip_special_tokens=True)
                print("RAW RESPONSE:\n", response, "\n---")

                person1_scores, person2_scores = extract_scores(response)
                if not person1_scores:
                    print("⚠️ Could not parse Person 1 scores.")

                # also write parsed scores explicitly
                print(f"Instruction: {instr}")
                print(f"Input: {ui}")
                print("Person 1 Scores:")
                for c in CHARACTERISTICS:
                    print(f"{c}: {person1_scores.get(c, 'N/A')}")
                if person2_scores:
                    print("\nPerson 2 Scores:")
                    for c in CHARACTERISTICS:
                        print(f"{c}: {person2_scores.get(c, 'N/A')}")
                else:
                    print("\n⚠️ No Person 2 scores parsed.")
                print("-" * 40)

            print("✅ Done. Results in", output_file)

        finally:
            # restore original stdout
            sys.stdout = original_stdout

# ——————————————————————————————
# Main
# ——————————————————————————————

if __name__ == "__main__":
    # 1) Load and quantize base model
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=MODEL_NAME,
        max_seq_length=2048,
        dtype=torch.float16,
        load_in_4bit=True,
        trust_remote_code=True
    )

    # 2) Register for inference
    FastLanguageModel.for_inference(model)

    # 3) Run inference
    process_test_json(TEST_JSON, OUTPUT_TXT, model, tokenizer, max_new_tokens=256)


Streaming output truncated to the last 5000 lines.
Calmness: 2
Politeness: 3
Aggression: 1
Sophistication: 1
Incoherence: 4
Rudeness: 2
Stupidity: 4
Articulation: 2
Unsophistication: 5
Laziness: 5

⚠️ No Person 2 scores parsed.
----------------------------------------
PROMPT:
 <|begin_of_text|> system: You are a helpful assistant, assisting with a research project on Standard American English and African American Vernacular English. <|eot_id|>
user: Analyze the type of person who would say "That all goes back to accepting accountability of one's outcome. They want the glorybut none of the liability". Evaluate them on the following characteristics with scores from 1-5. 1 = This person does not exhibit this characteristic. 2 = This person exhibits a small level of this characteristic. 3 = This person exhibits a moderate level of this characteristic. 4 = This person exhibits a high level of this characteristic. 5 = This person strongly and completely exhibits this characteristic. Only pro

In [ ]:
# Loading the fine-tuned model and the tokenizer for inference
model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = config.get("model_config").get("finetuned_model"),
        max_seq_length = config.get("model_config").get("max_seq_length"),
        dtype = config.get("model_config").get("dtype"),
        load_in_4bit = config.get("model_config").get("load_in_4bit"),
    )

# Using FastLanguageModel for fast inference
FastLanguageModel.for_inference(model)

#system_prompt = f"You are an AI task automator. You will take a users prompt and use first principle reasoning to break the prompt into tasks that you must accomplish within another chat. RESPOND TO THIS MESSAGE ONLY WITH A PYTHON FORMATTED LIST OF TASKS THAT YOU MUST COMPLETE TO TRUTHFULLY AND INTELLIGENTLY ACCOMPLISH THE USERS REQUEST. ASSUME YOU CAN SEARCH THE WEB, WRITE CODE, RUN CODE, DEBUG CODE, AND AUTOMATE ANYTHING ON THE USERS COMPUTER TO ACCOMPLISH THE PROMPT. CORRECT RESPONSE FORMAT: ['task 1', 'task 2', 'task 3']"
system_prompt = f"You are a helpful assistant, assisting with a research project on Standard American English and African American Vernacular English."

# Tokenizing the input and generating the output
prompt = input('TYPE PROMPT TO LLAMA3: ')
inputs = tokenizer(
[
    f"<|start_header_id|>system<|end_header_id|>{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>{prompt}<|end_header_id|>"
], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs, skip_special_tokens = True)

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 